## Mapping SH Clinic Data

In [ ]:
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
import pandas as pd
import pandas_profiling

%matplotlib inline

### Read in Cleaned Siloam Data 
(from 'Loading & Cleaning SH Data.ipynb' notebook)

In [ ]:
sh_df_geocoded = pd.read_csv('data/sh_df_geocoded.csv')
year_list = ['2011','2012','2013','2014','2015','2016','2017','2018']

In [ ]:
# location of SH clinic
sh_lat =  36.1187865
sh_lon = -86.778333

### Use folium with HeatMapWithTime to create heatmap of data per year of patient's first visit.

In [ ]:
# Use dataframe with all data, but drop records with null geolocation
df = sh_df_geocoded.dropna()

In [ ]:
hm_over_time = folium.Map(location = [sh_lat, sh_lon], zoom_start = 10.25)
map_data = hm_over_time

# Iterate over each year of geocoded data
heat_data = [[[row['latitude'],row['longitude']] for index,\
              row in df[df['Year_FirstVisit'].astype(str) == i ].iterrows()] for i in year_list]

# Plot data on map
hm = plugins.HeatMapWithTime(heat_data,auto_play=True,max_opacity=0.8)
hm.add_to(map_data)

# Add marker for SH Location
marker1 = folium.Marker(location = [sh_lat, sh_lon] 
                    , icon=folium.Icon(color = 'blue',icon_color='white',prefix='fa',icon='fa-plus-square'))
    
marker1.add_to(map_data)

print(map_data.save("Patient_Residency_HeatMap_Over_Time.html"))

### Look at folium HeatMap of active patients with visits in last 3 years.

In [ ]:
# Subset data to look at only active patients that have visited the clinic in the last 3 years 
RecentActivePatients = sh_df_geocoded.loc[(sh_df_geocoded['Year_LastVisit'].isin(['2016','2017','2018'])) \
                                          & (sh_df_geocoded['Active']=='Y')].dropna()

In [ ]:
df = RecentActivePatients
hm_active_patients = folium.Map(location = [sh_lat, sh_lon], zoom_start = 10.25)
map_data = hm_active_patients


heat_data = [[row['latitude'],row['longitude']] for index, row in df.iterrows()]

# Plot data on map
HeatMap(heat_data,min_opacity = 0.2).add_to(map_data)

# Add marker for SH location
marker0 = folium.Marker(location = [sh_lat, sh_lon] 
                    , icon=folium.Icon(color = 'blue',icon_color='white',prefix='fa',icon='fa-plus-square'))
    
marker0.add_to(map_data)

print(map_data.save("Patient_Residency_HeatMap_ActivePts.html"))

### Map data points of geolocation colored by distance bucket compared to both the SH & new site locations. 

In [ ]:
# Create color bands of data points based on distance bucket
def dist_color(value):
    if value == 'within 2':
        return '#37d9e8'
    elif value == 'within 5':
        return '#e89237'
    elif value == 'within 8':
        return '#a32175'
    elif value == 'within 10':
        return '#c6c676'
    elif value == 'within 15':
        return '#d17f7d'
    elif value == 'within 20':
        return '#0d94a3'
    else:
        return '#000080'

In [ ]:
# Apply dist_color function to both SH and NL buckets
sh_df_geocoded['SH_Dist_Color'] = sh_df_geocoded['SH_Dist_Bucket'].apply(lambda x: dist_color(x))
sh_df_geocoded['NL_Dist_Color'] = sh_df_geocoded['NL_Dist_Bucket'].apply(lambda x: dist_color(x))

In [ ]:
# coordinates of new location
nl_lat =  36.0704611
nl_lon = -86.6344346

In [ ]:
# Map SH Location and patient residencies colored by distance buckets
sh_map_by_dist = folium.Map(location = [sh_lat, sh_lon], zoom_start = 10.25)
map_data = sh_map_by_dist

# Maps colored data points of patient residences
for row in df.iterrows():
    row_values = row[1] 
    location = [row_values['latitude'], row_values['longitude']]
    marker = folium.CircleMarker(location=location
                                 ,color=row_values['SH_Dist_Color']
                                 ,radius=5
                                 ,fill=True
                                 ,fill_color=row_values['SH_Dist_Color']
                                 ,fill_opacity=0.8
                                 ,opacity=0.7)
    marker.add_to(map_data)

# Add marker for SH Location
marker0 = folium.Marker(location = [sh_lat, sh_lon] 
                    , icon=folium.Icon(color = 'blue',icon_color='white',prefix='fa',icon='fa-plus-square'))

marker0.add_to(map_data)

print(map_data.save("Patient_Residency_ActivePt_SH_Bucket_SH_marker_only.html"))

In [ ]:
# Map new site Location and patient residencies colored by distance buckets
nl_map_by_dist = folium.Map(location = [sh_lat, sh_lon], zoom_start = 10.25)
map_data = nl_map_by_dist

for row in df.iterrows():
    row_values = row[1] 
    location = [row_values['latitude'], row_values['longitude']]
    marker = folium.CircleMarker(location=location
                                 ,color=row_values['NL_Dist_Color']
                                 ,radius=5
                                 ,fill=True
                                 ,fill_color=row_values['NL_Dist_Color']
                                 ,fill_opacity=0.8
                                 ,opacity=0.7)
    marker.add_to(map_data)

# Add marker for SH Location
marker0 = folium.Marker(location = [sh_lat, sh_lon] 
                    , icon=folium.Icon(color = 'blue',icon_color='white',prefix='fa',icon='fa-plus-square'))

# Add marker for Potential New Location
marker1 = folium.Marker(location = [nl_lat, nl_lon] 
                    , icon=folium.Icon(color = 'green',icon_color='white',prefix='fa',icon='fa-plus-square'))

marker0.add_to(map_data)
marker1.add_to(map_data)

print(map_data.save("Patient_Residency_ActivePt_NL_Bucket.html"))

In [ ]:
# Number of patients in each bucket
print(RecentActivePatients['SH_Dist_Bucket'].value_counts())
print('----------------------------------')
print(RecentActivePatients['NL_Dist_Bucket'].value_counts())